# core

> Fill in a module description here

In [1]:
#| default_exp core

In [2]:
#| hide
from nbdev.showdoc import *

In [7]:
#| export
import toml
import os
from pathlib import Path
from dotenv import load_dotenv
load_dotenv()

def set_default_dir():
    print('Setting reproduce.work config dir to ./reproduce')
    return Path("./reproduce")

reproduce_dir = os.getenv("REPROWORKDIR", set_default_dir())
dev_image_tag = os.getenv("REPRODEVIMAGE")

def read_base_config():
    with open(Path(reproduce_dir, 'config.toml'), 'r') as f:
        base_config = toml.load(f)
    return base_config



def validate_base_config(base_config):
    required_keys = ['authors', 'repro']
    for key in required_keys:
        if key not in base_config:
            print(toml.dumps(base_config))
            print(f"Error: Missing required field '{key}' in config.toml")
            return False
        if key=='repro':
            if 'stages' not in base_config['repro']:
                print(f"Error: Missing required field 'repro.stages' in reproduce.work configuration at {reproduce_dir}/config.toml")
                return False
            for stage in base_config['repro']['stages']:
                if (f'repro.stage.{stage}' not in base_config) and (stage not in base_config['repro']['stage']):
                    print(toml.dumps(base_config))
                    print(f"Error: Missing required field repro.stage.{stage} in reproduce.work configuration at {reproduce_dir}/config.toml")
                    return False
    return True

def requires_config(func):
    def wrapper(*args, **kwargs):
        config = read_base_config()
        if not validate_base_config(config):
            raise Exception("Your reproduce.work configuration is not valid.")
        return func(*args, **kwargs)
    return wrapper


class ReproduceWorkEncoder(toml.TomlEncoder):
    def dump_str(self, v):
        """Encode a string."""
        if "\n" in v:
            return v  # If it's a multi-line string, return it as-is
        return super().dump_str(v)
    
    def dump_value(self, v):
        """Determine the type of a Python object and serialize it accordingly."""
        if isinstance(v, str) and "\n" in v:
            return '"""\n' + v.strip() + '\n' + '"""'
        return super().dump_value(v)


def generate_config(inputs={}, version="reproduce.work/v1/default"):
    if inputs=={}:
        # Authors section
        author1_email = input("Enter author's email (required): ")
        author1_name = input("Enter author's name: ")
        author1_affiliation = input("Enter author's affiliation: ")
        repro_version = input(f"Enter reproduce_work API version (Default: {version}): ") or version

        # Repro stage init section
        dev_image_tag = input("Enter dev image tag (required; the docker image of your local development workflow): ")
        
        # default should be no
        nbdev_project = input("Is this a nbdev project? (y/n) ") == 'y'
        github_repo = input("Enter github repo (required): ")

        project_full_title = input("Enter project full title (options): ") or "Title goes here"
        project_abstract = input("Enter project abstract (optional): ") or "Abstract goes here."
    
    else:
        author1_email = inputs['authors']['author1']['email']
        if 'name' in inputs['authors']['author1']:
            author1_name = inputs['authors']['author1']['name']
        else:
            author1_name = None

        if 'affiliation' in inputs['authors']['author1']:
            author1_affiliation = inputs['authors']['author1']['affiliation']
        else:
            author1_affiliation = None
            
        dev_image_tag = inputs['dev_image_tag']
        repro_version = version

        nbdev_project = False
        if 'nbdev_project' in inputs:
            nbdev_project = inputs['nbdev_project']


        project_full_title = 'Title goes here'
        project_abstract = 'Abstract goes here.'
        github_repo = False
        if 'project' in inputs:
            if 'github_repo' in inputs['project']:
                github_repo = inputs['project']['github_repo']
            if 'full_title' in inputs['project']:
                project_full_title = inputs['project']['full_title']
            if 'abstract' in inputs['project']:
                project_abstract = inputs['project']['abstract']
        
        verbose = False
        if 'verbose' in inputs:
            verbose = inputs['verbose']

    verbose_str = "true" if verbose else "false"

    if repro_version in ["reproduce.work/v1/default"]:
        document_dir = reproduce_dir
        input_file = f"{document_dir}/main.md"
        dynamic_file = f"{document_dir}/pubdata.toml"
        latex_template = f"{document_dir}/latex/template.tex"
        output_file =f"{document_dir}/latex/compiled.tex"
        watch_files = [input_file, dynamic_file, latex_template]
    
    # Check for critical fields
    if not dev_image_tag:
        print("Error: Missing critical field 'dev_image_tag'.")
        return
    
    nbdev_project_cmd = ""
    if nbdev_project:
        nbdev_project_cmd = '\nnbdev_install_hooks && nbdev_export'

    github_repo_str = ''
    if github_repo:
        github_repo_str = f'\ngithub_repo = "{github_repo}"'

    #reproduce_dir_default = f"{reproduce_dir}"  # this can be changed if needed
    #reproduce_dir = input(f"Enter reproduce directory (Default: {reproduce_dir_default}): ") or reproduce_dir_default
    
    repro_init_script = f'''docker build -t {dev_image_tag} .
    docker build -t tex-prepare https://github.com/reproduce-work/tex-prepare.git
    docker build -t tex-compile https://github.com/reproduce-work/tex-compile.git
    docker build -t watcher https://github.com/reproduce-work/rwatch.git
    '''

    # ensure reproduce_dir exists
    Path(reproduce_dir).mkdir(parents=True, exist_ok=True)

    newline = "\n"
    with open(Path(reproduce_dir, 'config.toml'), 'w') as f:
        f.write(f'''
[authors]
author1.email = "{author1_email}"{newline + 'author1.name = "' + author1_name + '"' if author1_name else ""}{newline + 'author1.affiliation = "' + author1_affiliation + '"' if author1_affiliation else ""}

[project]
full_title = "{project_full_title}"
abstract = """
{project_abstract}
"""{github_repo_str}

# reproduce.work configuration
[repro]
version = "{repro_version}"
stages = ["init", "develop", "build"]
verbose = {verbose_str}
terminal_linefile = "report.tex" # must be plaintext file
terminal_file = "report.pdf" # can be any filetype

[repro.files]
input = "{input_file}"
dynamic = "{dynamic_file}"
latex_template = "{latex_template}"
output = "{output_file}"
watch = {watch_files}

[repro.stage.init]
script = """
{repro_init_script}
"""

[repro.stage.develop]
script = """
docker run -v $(pwd):/home/jovyan -p 8888:8888 {dev_image_tag}
\INSERT{{watch_cmd_here}}
"""

[repro.stage.build]
script = """
docker run --rm -i -v $(pwd):/home/jovyan -p 8888:8888 {dev_image_tag} python reproduce_work.build() # this replaces instances of INSERTvar in input file
docker run --rm -i -v $(pwd):/home -e REPROWORKDIR="{reproduce_dir}" -e REPROWORKOUTFILE="{output_file}" tex-prepare python build.py # this converts the markdown to latex
docker run --rm -i --net=none -v $(pwd):/home tex-compile sh -c "cd /home/{document_dir}/latex && xelatex compiled.tex" # this compiles the latex{nbdev_project_cmd}
"""
        ''')


    # By default, TOML files cannot reference variables defined earlier in the file
    # This is a hack to get around that which allows me to read in the TOML data,
    # and then backfill the watch command with the files listed in the TOML file itself.
    if '\INSERT{watch_cmd_here}' in open(Path(reproduce_dir, 'config.toml'), 'r').read():
        with open(Path(reproduce_dir, 'config.toml'), 'r') as f:
            base_config_str = f.read()
        replaced_config = base_config_str.replace(
            '\INSERT{watch_cmd_here}', 
            f"""docker run watcher "{','.join(watch_files)}" "echo \"File has changed!\" && build_cmd" """
        )
        with open(Path(reproduce_dir, 'config.toml'), 'w') as f:
            f.write(replaced_config)
    else:
        print('did not replace watch command')


    with open(Path(reproduce_dir, 'config.toml'), 'r') as f:
        config_data = toml.load(f)

    with open(Path(reproduce_dir, 'config.toml'), 'w') as f:
        toml.dump(config_data, f, encoder=ReproduceWorkEncoder())

    print(f"Successfully generated reproduce.work configuration at {reproduce_dir}/config.toml")

    # check if input file exists
    if not os.path.exists(input_file):
        with open(input_file, 'w') as f:
            f.write('')
        print(f"Successfully generated input file at {input_file}")

    # check if latex template exists
    if not os.path.exists(latex_template):
        # Fill in with package default
        Path(latex_template).parent.mkdir(parents=True, exist_ok=True)
        latex_template_default_str = r'\documentclass[12pt]{article}\usepackage[english]{babel}\usepackage{xcolor}\usepackage[hmargin=1in,vmargin=1in]{geometry}\usepackage{amsmath}\usepackage{unicode-math}\usepackage[round,sort,comma]{natbib}\bibliographystyle{apa}\usepackage{setspace}\usepackage{graphicx}\usepackage{caption}\usepackage{subcaption}\usepackage[colorlinks=true, allcolors=blue]{hyperref}\usepackage{float}\usepackage{booktabs}\usepackage{titlesec}\newcommand{\addperiod}[1]{#1.$\;$}\titlespacing{\section}{0pt}{\parskip}{-\parskip}\titleformat{\subsection}[runin]{\normalsize\bfseries}{\thesubsection}{1em}{\addperiod}\titleformat{\subsubsection}[runin]{\normalfont\normalsize\itshape}{\thesubsubsection}{1em}{\addperiod}\titlespacing{\subsubsection}{14pt plus 4pt minus 2pt}{0pt}{0pt plus 2pt minus 2pt}\setlength{\parindent}{1em}\makeatletter\g@addto@macro \normalsize {\setlength\abovedisplayskip{3pt plus 5pt minus 2pt}\setlength\belowdisplayskip{3pt plus 5pt minus 2pt}}\makeatother\newcommand{\comment}[1]{}\begin{document}\pagenumbering{gobble}\begin{center}{\fontsize{16}{16}\selectfont\bfseries \INSERT{config.project.full_title}}\vspace{5mm}\begin{table}[!ht]\begin{center}\begin{tabular}{c c }\shortstack{ \INSERT{config.authors.author1.name} \\\INSERT{config.authors.author1.affiliation} \\\INSERT{config.authors.author1.email} }\end{tabular}\end{center}\end{table}\vspace{5mm}\emph{Last updated: \today}\vspace{4mm}\abstract{\INSERT{config.project.abstract}}\vspace{2cm}{\scriptsize \noindent Notes: }\vspace{10mm}\end{center}\newpage\doublespacing\pagenumbering{arabic}\setcounter{page}{1}%%@@LOWDOWN_CONTENT@@%%\bibliography{latex/bibliography}\end{document}'
        with open(latex_template, 'w') as f:
            f.write(latex_template_default_str)
        print(f"Successfully generated latex template at {latex_template}")


# Write some basic tests
def test_validate_base_config():
    # Test a valid base_config
    base_config = {
        'authors': {
            'author1.name': 'Alex P. Miller',
            'author1.affiliation': 'USC Marshall School of Business',
            'author1.email': 'alex.miller@marshall.usc.edu'
        },
        'repro': {
            'version': 'reproduce.work/v1/default',
            'stages': ['init', 'develop', 'build']
        },
        'repro.files': {
            'input': 'document/main.md',
            'dynamic': 'document/pubdata.toml',
            'latex_template': 'document/latex/template.tex',
            'output': 'document/latex/compiled.tex',
            'watch': ['document/main.md', 'document/pubdata.toml', 'document/latex/template.tex']
        },
        'repro.stage.init': {
            'script': 'docker build -t {dev_image_tag} .\ndocker build -t tinytex {reproduce_dir}/Dockerfile.tinytex\ndocker build -t watcher {reproduce_dir}/Dockerfile.watch\n'
        },
        'repro.stage.develop': {
            'script': 'docker run -v $(pwd):/home/jovyan -p 8888:8888 {dev_image_tag} start.sh jupyter lab --LabApp.token=\'\'\n\\INSERT{watch_cmd_here}\n'
        },
        'repro.stage.build': {
            'script': 'docker run --rm -i -v $(pwd):/home/jovyan -p 8888:8888 {dev_image_tag} python reproduce_work.build() # this replaces instances of \\INSERT{var} in `input` file\ndocker run --rm -i -v $(pwd):/home lowdown # this converts the markdown to latex\ndocker run --rm -i --net=none -v $(pwd):/home tinytex sh -c "cd /home/document/latex && xelatex compiled.tex" # this compiles the latex\n'
        }
    }
    assert validate_base_config(base_config) == True

# Test an invalid base_config
base_config = {
    'au': {
        'author1.name': 'Alex P. Miller',
        'author1.affiliation': 'USC Marshall School of Business',
        'author1.email': 'alex.miller@marshall.usc.edu'
    },
    'repro': {
        'version': 'reproduce.work/v1/default',
        'stages': ['init', 'develop', 'build']
    },
    'repro.files': {
        'input': 'document/main.md',
        'dynamic': 'document/pubdata.toml',
        'latex_template': 'document/latex/template.tex',
        'output': 'document/latex/compiled.tex'
    },
    'repro.stage.init': {
        'script': 'docker build -t {dev_image_tag} .\ndocker build -t tinytex {reproduce_dir}/Dockerfile.tinytex\ndocker build -t watcher {reproduce_dir}/Dockerfile.watch\n'
    },
    'repro.stage.develop': {
        'script': 'docker run -v $(pwd):/home/jovyan -p 8888:8888 {dev_image_tag} start.sh jupyter lab --LabApp.token=\'\'\n\\INSERT{watch_cmd_here}\n'
    },
    'repro.stage.build': {
        'script': 'docker run --rm -i -v $(pwd):/home/jovyan -p 8888:8888 {dev_image_tag} python reproduce_work.build() # this replaces instances of \\INSERT{var} in `input` file\ndocker run --rm -i -v $(pwd):/home lowdown # this converts the markdown to latex\ndocker run --rm -i --net=none -v $(pwd):/home tinytex sh -c "cd /home/document/latex && xelatex compiled.tex" # this compiles the latex\n'
    }
}
assert validate_base_config(base_config) == False

print("All tests passed!")



Setting reproduce.work config dir to ./reproduce
[au]
"author1.name" = "Alex P. Miller"
"author1.affiliation" = "USC Marshall School of Business"
"author1.email" = "alex.miller@marshall.usc.edu"

[repro]
version = "reproduce.work/v1/default"
stages = [ "init", "develop", "build",]

["repro.files"]
input = "document/main.md"
dynamic = "document/pubdata.toml"
latex_template = "document/latex/template.tex"
output = "document/latex/compiled.tex"

["repro.stage.init"]
script = "docker build -t {dev_image_tag} .\ndocker build -t tinytex {reproduce_dir}/Dockerfile.tinytex\ndocker build -t watcher {reproduce_dir}/Dockerfile.watch\n"

["repro.stage.develop"]
script = "docker run -v $(pwd):/home/jovyan -p 8888:8888 {dev_image_tag} start.sh jupyter lab --LabApp.token=''\n\\INSERT{watch_cmd_here}\n"

["repro.stage.build"]
script = "docker run --rm -i -v $(pwd):/home/jovyan -p 8888:8888 {dev_image_tag} python reproduce_work.build() # this replaces instances of \\INSERT{var} in `input` file\ndocker 

In [4]:
#import os
#os.listdir(os.getcwd())

In [8]:
base_config = {
        'authors': {
            'author1.name': 'Alex P. Miller',
            'author1.affiliation': 'USC Marshall School of Business',
            'author1.email': 'alex.miller@marshall.usc.edu'
        },
        'repro': {
            'version': 'reproduce.work/v1/default',
            'stages': ['init', 'develop', 'build']
        },
        'repro.files': {
            'input': 'document/main.md',
            'dynamic': 'document/pubdata.toml',
            'latex_template': 'document/latex/template.tex',
            'output': 'document/latex/compiled.tex'
        },
        'repro.stage.init': {
            'script': 'docker build -t {dev_image_tag} .\ndocker build -t tinytex {reproduce_dir}/Dockerfile.tinytex\ndocker build -t watcher {reproduce_dir}/Dockerfile.watch\n'
        },
        'repro.stage.develop': {
            'script': 'docker run -v $(pwd):/home/jovyan -p 8888:8888 {dev_image_tag} start.sh jupyter lab --LabApp.token=\'\'\n\\INSERT{watch_cmd_here}\n'
        },
        'repro.stage.build': {
            'script': 'docker run --rm -i -v $(pwd):/home/jovyan -p 8888:8888 {dev_image_tag} python reproduce_work.build() # this replaces instances of \\INSERT{var} in `input` file\ndocker run --rm -i -v $(pwd):/home lowdown # this converts the markdown to latex\ndocker run --rm -i --net=none -v $(pwd):/home tinytex sh -c "cd /home/document/latex && xelatex compiled.tex" # this compiles the latex\n'
        }
    }
validate_base_config(base_config)

True

In [9]:
#| hide
import nbdev; nbdev.nbdev_export()